In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# John Hopkins University Country Data
jhu_csv = 'data/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_%s_global.csv'
# New York Times State/County Data
nyt_csv = 'data/covid-19-data/us-%s.csv'

data = pd.read_csv(nyt_csv % 'states')

grouped = data.groupby(['state'])

fig, ax = plt.subplots(figsize=(15,7))

for state, item in grouped:
    if state not in ['Washington', 'Florida', 'New York']:
        continue
    df = item.copy()
    df.loc[:, 'cases_delta'] = df['cases'].diff(1)
    df.loc[:, 'rolling_cases'] = df.cases.rolling(7).mean()
    df.loc[:, 'rolling_cases_delta'] = df.cases_delta.rolling(7).mean()
    df.plot(ax=ax, label=state, x='rolling_cases', y='rolling_cases_delta', logx=True, logy=True)
    plt.text(df.rolling_cases.max(), df.rolling_cases_delta.max(), state)
    

ax.set_xlabel('Log of Total Cases')
ax.set_ylabel('Log of New Cases')